In [2]:
# Simple Knowledge Representation using facts + rules
# Facts are stored as (subject, relation, object) tuples
# Rules infer new facts from existing ones (forward chaining)

from dataclasses import dataclass
from typing import Callable, Set, Tuple, List

# A fact is a triple: (subject, relation, object)
Fact = Tuple[str, str, str]


# Rule structure: a name + a function that derives new facts
@dataclass(frozen=True)
class Rule:
    name: str
    apply: Callable[[Set[Fact]], Set[Fact]]


class KnowledgeBase:
    def __init__(self):
        self.facts: Set[Fact] = set()   # known facts
        self.rules: List[Rule] = []     # inference rules

    def add_fact(self, fact: Fact):
        # Add a fact (duplicates automatically ignored)
        self.facts.add(fact)

    def add_rule(self, rule: Rule):
        # Add a reasoning rule
        self.rules.append(rule)

    def infer(self):
        # Apply rules repeatedly until no new facts are found
        while True:
            new_facts = set()

            for rule in self.rules:
                inferred = rule.apply(self.facts)
                new_facts |= (inferred - self.facts)

            if not new_facts:
                break  # stop when nothing new is inferred

            self.facts |= new_facts

    def query(self, subject=None, relation=None, obj=None):
        # Simple pattern-based query
        return [
            f for f in self.facts
            if (subject is None or f[0] == subject)
            and (relation is None or f[1] == relation)
            and (obj is None or f[2] == obj)
        ]


# ---------- RULE DEFINITIONS ----------

def parent_implies_child(facts: Set[Fact]) -> Set[Fact]:
    # If X parent_of Y, then Y child_of X
    return {(o, "child_of", s) for (s, r, o) in facts if r == "parent_of"}


def parent_implies_ancestor(facts: Set[Fact]) -> Set[Fact]:
    # If X parent_of Y, then X ancestor_of Y
    return {(s, "ancestor_of", o) for (s, r, o) in facts if r == "parent_of"}


def ancestor_transitive(facts: Set[Fact]) -> Set[Fact]:
    # If X ancestor_of Y and Y ancestor_of Z, then X ancestor_of Z
    ancestors = [(s, o) for (s, r, o) in facts if r == "ancestor_of"]
    return {
        (x, "ancestor_of", z)
        for (x, y) in ancestors
        for (y2, z) in ancestors
        if y == y2 and x != z
    }


# ---------- BUILD KNOWLEDGE BASE ----------

kb = KnowledgeBase()

# Base facts
kb.add_fact(("john", "parent_of", "mary"))
kb.add_fact(("mary", "parent_of", "ali"))
kb.add_fact(("ali", "parent_of", "zara"))

# Rules
kb.add_rule(Rule("parent -> child", parent_implies_child))
kb.add_rule(Rule("parent -> ancestor", parent_implies_ancestor))
kb.add_rule(Rule("ancestor transitive", ancestor_transitive))

# Run inference
kb.infer()

# ---------- QUERIES ----------

print("All facts:")
for fact in sorted(kb.facts):
    print(fact)

print("\nAncestors of zara:")
print(kb.query(relation="ancestor_of", obj="zara"))

All facts:
('ali', 'ancestor_of', 'zara')
('ali', 'child_of', 'mary')
('ali', 'parent_of', 'zara')
('john', 'ancestor_of', 'ali')
('john', 'ancestor_of', 'mary')
('john', 'ancestor_of', 'zara')
('john', 'parent_of', 'mary')
('mary', 'ancestor_of', 'ali')
('mary', 'ancestor_of', 'zara')
('mary', 'child_of', 'john')
('mary', 'parent_of', 'ali')
('zara', 'child_of', 'ali')

Ancestors of zara:
[('john', 'ancestor_of', 'zara'), ('mary', 'ancestor_of', 'zara'), ('ali', 'ancestor_of', 'zara')]
